Import tools.

In [2]:
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.ensemble import VotingClassifier
from sklearn.feature_selection import RFECV
from sklearn.pipeline import Pipeline
from pipelinehelper import PipelineHelper
from sklearn.preprocessing import MaxAbsScaler

In [3]:
import pandas as pd
import numpy as np
from pickle import load
from pickle import dump
pd.set_option("max_columns", None)
pd.set_option('max_rows', None)
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.metrics import accuracy_score, make_scorer, f1_score, precision_score, recall_score

Bring in dataframe that has race, poverty, and results joined and has Alaska all tidied up.

In [4]:
df = load(open('df_all.pkl', 'rb'))

Look at and drop county and all but one state column for modeling. Drop id.

In [5]:
df = df.drop(['County_x', 'id', 'State_x', 'County_y', 'State_y', 'county'], axis = 1)

In [6]:
state_dummies = pd.get_dummies(df['state'], drop_first = True)
central_outlying = pd.get_dummies(df['central_outlying'], drop_first = True)

In [7]:
df = df.drop(['state', 'central_outlying'], axis = 1)

In [8]:
df = pd.concat([df, state_dummies, central_outlying], axis = 1)

In [9]:
df.Target = df.Target.map({'Trump': 0, 'Clinton': 1})

In [10]:
X = df.drop(['Target'], axis = 1)
y = df.Target

In [11]:
f1_scores = make_scorer(f1_score)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, random_state = 42)

In [22]:
y_train.head()

1919    0
37      0
2742    0
1478    1
56      1
Name: Target, dtype: int64

In [13]:
X_train.isna().sum()

total_pop                  0
total_pop_one_race         0
pop_white                  0
pop_african_american       0
pop_native                 0
pop_asian                  0
pop_islander               0
pop_other                  0
total_pop_two_races        0
2016_total_votes           0
Obama                      0
Romney                     0
2012_total_votes           0
2010_land_area             0
Density                    0
poverty_total              0
poverty_under_18           0
median_household_income    0
Alaska                     0
Arizona                    0
Arkansas                   0
California                 0
Colorado                   0
Connecticut                0
Delaware                   0
District of Columbia       0
Florida                    0
Georgia                    0
Hawaii                     0
Idaho                      0
Illinois                   0
Indiana                    0
Iowa                       0
Kansas                     0
Kentucky      

In [12]:
#         ('lda', LinearDiscriminantAnalysis()),
#         ('qda', QuadraticDiscriminantAnalysis()),
#         ('KNN', KNeighborsClassifier()),
#         ('bayes', GaussianNB()),

In [23]:
pipe

Pipeline(steps=[('scaler',
                 PipelineHelper(available_models={'max': MaxAbsScaler(),
                                                  'minmax': MinMaxScaler(),
                                                  'std': StandardScaler()})),
                ('classifier',
                 PipelineHelper(available_models={'KNN': KNeighborsClassifier(),
                                                  'bayes': GaussianNB(),
                                                  'lda': LinearDiscriminantAnalysis(),
                                                  'qda': QuadraticDiscriminantAnalysis()}))])

In [14]:
pipe = Pipeline([
    ('scaler', PipelineHelper([
        ('std', StandardScaler()),
        ('max', MaxAbsScaler()),
        ('minmax', MinMaxScaler())
    ])),
    ('classifier', PipelineHelper([
        ('lda', LinearDiscriminantAnalysis()),
        ('qda', QuadraticDiscriminantAnalysis()),
        ('KNN', KNeighborsClassifier()),
        ('bayes', GaussianNB())
    ])),
])

In [15]:
params = {
    'scaler__selected_model': pipe.named_steps['scaler'].generate({
        'std__with_mean': [True, False],
        'std__with_std': [True, False],
        'max__copy': [True],  # just for displaying
    }),
    'classifier__selected_model': pipe.named_steps['classifier'].generate({
        'lda__solver': ['svd', 'lsqr', 'eigen'],
    })
}

In [16]:
grid = GridSearchCV(pipe, params, scoring= f1_scores)

In [17]:
grid.fit(X_train, y_train)

C:\Users\angie\anaconda3\envs\learn-env\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\angie\anaconda3\envs\learn-env\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\angie\anaconda3\envs\learn-env\lib\site-packages\sklearn\pipeline.py", line 335, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\angie\anaconda3\envs\learn-env\lib\site-packages\pipelinehelper\__init__.py", line 159, in fit
    return self.selected_model.fit(x, y, **kwargs)
  File "C:\Users\angie\anaconda3\envs\learn-env\lib\site-packages\sklearn\discriminant_analysis.py", line 468, in fit
    self._solve_eigen(X, y, shrinkage=self.shrinkage)
  File "C:\Users\angie\anaconda3\envs\learn-env

C:\Users\angie\anaconda3\envs\learn-env\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\angie\anaconda3\envs\learn-env\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\angie\anaconda3\envs\learn-env\lib\site-packages\sklearn\pipeline.py", line 335, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\angie\anaconda3\envs\learn-env\lib\site-packages\pipelinehelper\__init__.py", line 159, in fit
    return self.selected_model.fit(x, y, **kwargs)
  File "C:\Users\angie\anaconda3\envs\learn-env\lib\site-packages\sklearn\discriminant_analysis.py", line 468, in fit
    self._solve_eigen(X, y, shrinkage=self.shrinkage)
  File "C:\Users\angie\anaconda3\envs\learn-env

C:\Users\angie\anaconda3\envs\learn-env\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\angie\anaconda3\envs\learn-env\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\angie\anaconda3\envs\learn-env\lib\site-packages\sklearn\pipeline.py", line 335, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\angie\anaconda3\envs\learn-env\lib\site-packages\pipelinehelper\__init__.py", line 159, in fit
    return self.selected_model.fit(x, y, **kwargs)
  File "C:\Users\angie\anaconda3\envs\learn-env\lib\site-packages\sklearn\discriminant_analysis.py", line 468, in fit
    self._solve_eigen(X, y, shrinkage=self.shrinkage)
  File "C:\Users\angie\anaconda3\envs\learn-env

C:\Users\angie\anaconda3\envs\learn-env\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\angie\anaconda3\envs\learn-env\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\angie\anaconda3\envs\learn-env\lib\site-packages\sklearn\pipeline.py", line 335, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\angie\anaconda3\envs\learn-env\lib\site-packages\pipelinehelper\__init__.py", line 159, in fit
    return self.selected_model.fit(x, y, **kwargs)
  File "C:\Users\angie\anaconda3\envs\learn-env\lib\site-packages\sklearn\discriminant_analysis.py", line 468, in fit
    self._solve_eigen(X, y, shrinkage=self.shrinkage)
  File "C:\Users\angie\anaconda3\envs\learn-env

C:\Users\angie\anaconda3\envs\learn-env\lib\site-packages\sklearn\discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\angie\anaconda3\envs\learn-env\lib\site-packages\sklearn\discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\angie\anaconda3\envs\learn-env\lib\site-packages\sklearn\discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\angie\anaconda3\envs\learn-env\lib\site-packages\sklearn\discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\angie\anaconda3\envs\learn-env\lib\site-packages\sklearn\discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\angie\anaconda3\envs\learn-env\lib\site-packages\sklearn\discriminant_analysis.py:715: UserWarning: Variables a

GridSearchCV(estimator=Pipeline(steps=[('scaler',
                                        PipelineHelper(available_models={'max': MaxAbsScaler(),
                                                                         'minmax': MinMaxScaler(),
                                                                         'std': StandardScaler()})),
                                       ('classifier',
                                        PipelineHelper(available_models={'KNN': KNeighborsClassifier(),
                                                                         'bayes': GaussianNB(),
                                                                         'lda': LinearDiscriminantAnalysis(),
                                                                         'qda': QuadraticDiscriminantAnalysis()}))]),
             param_grid={'classifie...
                                                         {'solver': 'lsqr'}),
                                                       

In [18]:
grid.best_score_

0.633152402503659

In [19]:
grid.best_params_

{'classifier__selected_model': ('KNN', {}),
 'scaler__selected_model': ('std', {'with_mean': True, 'with_std': False})}

In [20]:
grid.best_estimator_

Pipeline(steps=[('scaler',
                 PipelineHelper(available_models={'max': MaxAbsScaler(),
                                                  'minmax': MinMaxScaler(),
                                                  'std': StandardScaler(with_std=False)},
                                selected_model=StandardScaler(with_std=False))),
                ('classifier',
                 PipelineHelper(available_models={'KNN': KNeighborsClassifier(),
                                                  'bayes': GaussianNB(),
                                                  'lda': LinearDiscriminantAnalysis(),
                                                  'qda': QuadraticDiscriminantAnalysis()},
                                selected_model=KNeighborsClassifier()))])

In [21]:
y_hat_train = cross_val_predict(pipe, X_train, y_train)

AttributeError: 'NoneType' object has no attribute 'predict'

In [ ]:
confusion_matrix(y_train, y_hat_train)